In [1]:
import seaborn as sns
sns.set_palette("husl")
import matplotlib.pyplot as plt
%matplotlib inline
import hopsworks
import pandas as pd
import getpass


c:\Users\saadg\Desktop\kth-\classes\smldl\Scalable-ML\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = getpass.getpass(prompt="Enter your Hopsworks API key: ")

In [3]:
hopsworks.logout()

In [4]:
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/218261
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
wine_df = pd.read_csv(
    "https://raw.githubusercontent.com/ID2223KTH/id2223kth.github.io/master/assignments/lab1/wine.csv"
)
wine_df

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [6]:
wine_df.quality.value_counts()

quality
6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: count, dtype: int64

In [7]:
num_cols = wine_df.select_dtypes(include=["float", "int"]).columns
for c in num_cols:
    wine_df[c].fillna(wine_df[c].mean(), inplace=True)

In [ ]:
# color = sns.color_palette("pastel")

# fig, ax1 = plt.subplots(3, 4, figsize=(24, 30))
# k = 0
# columns = list(wine_df.columns)
# for i in range(3):
#     for j in range(4):
#         sns.distplot(wine_df[num_cols[k]], ax=ax1[i][j], color="red")
#         k += 1
# plt.show()

In [8]:
import numpy as np


def log_transform(col):
    return np.log(col[0])


wine_df["residual sugar"] = wine_df[["residual sugar"]].apply(log_transform, axis=1)
wine_df["chlorides"] = wine_df[["chlorides"]].apply(log_transform, axis=1)
wine_df["free sulfur dioxide"] = wine_df[["free sulfur dioxide"]].apply(
    log_transform, axis=1
)
wine_df["sulphates"] = wine_df[["sulphates"]].apply(log_transform, axis=1)

In [9]:
wine_df[num_cols].corr()["quality"].sort_values(ascending=False)

quality                 1.000000
alcohol                 0.444319
free sulfur dioxide     0.113564
citric acid             0.085699
sulphates               0.027653
pH                      0.019356
residual sugar         -0.012731
total sulfur dioxide   -0.041385
fixed acidity          -0.076967
volatile acidity       -0.265838
chlorides              -0.267684
density                -0.305858
Name: quality, dtype: float64

In [10]:
wine_df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.216579,0.339691,0.318722,1.326036,-2.990434,3.216126,115.744574,0.994697,3.218395,-0.666902,10.491801,5.818378
std,1.295751,0.164548,0.145231,0.863482,0.431055,0.698635,56.521855,0.002999,0.160637,0.256990,1.192712,0.873255
min,3.800000,0.080000,0.000000,-0.510826,-4.710531,0.000000,6.000000,0.987110,2.720000,-1.514128,8.000000,3.000000
25%,6.400000,0.230000,0.250000,0.587787,-3.270169,2.833213,77.000000,0.992340,3.110000,-0.843970,9.500000,5.000000
50%,7.000000,0.290000,0.310000,1.098612,-3.057608,3.367296,118.000000,0.994890,3.210000,-0.673345,10.300000,6.000000
75%,7.700000,0.400000,0.390000,2.091864,-2.733368,3.713572,156.000000,0.996990,3.320000,-0.510826,11.300000,6.000000
max,15.900000,1.580000,1.660000,4.186620,-0.492658,5.666427,440.000000,1.038980,4.010000,0.693147,14.900000,9.000000


In [11]:
import numpy as np
from scipy import stats


def detect_outliers(data):
    z = np.abs(stats.zscore(data))
    return z > 3.5


outlier_indices = []

for col in num_cols:
    # Boolean array of outlier values
    outliers = detect_outliers(wine_df[col])

    # Append outlier indexes to master list
    outlier_indices.extend(wine_df[outliers].index)

# Drop duplicates
outlier_indices = list(set(outlier_indices))

# Remove outliers
wine_df.drop(outlier_indices, inplace=True)

In [12]:
wine_df.describe(include="all")

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6287,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000
unique,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,white,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,4855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,7.134470,0.332135,0.314569,1.336299,-3.011722,3.237369,117.246700,0.994593,3.218784,-0.675817,10.494835,5.828535
std,NaN,1.125014,0.153446,0.137564,0.867543,0.400443,0.683444,55.534623,0.002893,0.157699,0.247957,1.194595,0.863326
min,NaN,3.800000,0.080000,0.000000,-0.510826,-4.422849,1.098612,6.000000,0.987110,2.720000,-1.514128,8.000000,3.000000
25%,NaN,6.400000,0.230000,0.250000,0.587787,-3.270169,2.833213,81.000000,0.992250,3.110000,-0.843970,9.500000,5.000000
50%,NaN,6.900000,0.290000,0.310000,1.163151,-3.057608,3.367296,119.000000,0.994800,3.210000,-0.693147,10.300000,6.000000
75%,NaN,7.600000,0.400000,0.390000,2.104134,-2.780621,3.737670,156.000000,0.996850,3.320000,-0.510826,11.300000,6.000000


In [13]:
wine_df.quality.value_counts()

quality
6    2755
5    2062
7    1059
4     198
8     192
3      21
Name: count, dtype: int64

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
wine_df["type"] = le.fit_transform(wine_df["type"])

In [15]:
wine_df.describe(include="all")

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000,6287.000000
mean,0.772228,7.134470,0.332135,0.314569,1.336299,-3.011722,3.237369,117.246700,0.994593,3.218784,-0.675817,10.494835,5.828535
std,0.419428,1.125014,0.153446,0.137564,0.867543,0.400443,0.683444,55.534623,0.002893,0.157699,0.247957,1.194595,0.863326
min,0.000000,3.800000,0.080000,0.000000,-0.510826,-4.422849,1.098612,6.000000,0.987110,2.720000,-1.514128,8.000000,3.000000
25%,1.000000,6.400000,0.230000,0.250000,0.587787,-3.270169,2.833213,81.000000,0.992250,3.110000,-0.843970,9.500000,5.000000
50%,1.000000,6.900000,0.290000,0.310000,1.163151,-3.057608,3.367296,119.000000,0.994800,3.210000,-0.693147,10.300000,6.000000
75%,1.000000,7.600000,0.400000,0.390000,2.104134,-2.780621,3.737670,156.000000,0.996850,3.320000,-0.510826,11.300000,6.000000
max,1.000000,11.700000,0.915000,0.820000,3.260018,-1.487220,4.987025,313.000000,1.003690,3.780000,0.198851,14.200000,8.000000


In [16]:
wine_df_3 = wine_df[wine_df.quality == 3]  # MINORITY
wine_df_4 = wine_df[wine_df.quality == 4]  # MINORITY
wine_df_5 = wine_df[wine_df.quality == 5]  # MAJORITY
wine_df_6 = wine_df[wine_df.quality == 6]  # MAJORITY
wine_df_7 = wine_df[wine_df.quality == 7]  # MINORITY
wine_df_8 = wine_df[wine_df.quality == 8]  # MINORITY

In [17]:
from sklearn.utils import resample

df_3_upsampled = resample(wine_df_3, replace=True, n_samples=2000, random_state=12)
df_4_upsampled = resample(wine_df_4, replace=True, n_samples=2000, random_state=12)
df_7_upsampled = resample(wine_df_7, replace=True, n_samples=2000, random_state=12)
df_8_upsampled = resample(wine_df_8, replace=True, n_samples=2000, random_state=12)

# Decreases the rows of Majority one's to make balance data :
df_5_downsampled = wine_df[wine_df.quality == 5].sample(n=2000).reset_index(drop=True)
df_6_downsampled = wine_df[wine_df.quality == 6].sample(n=2000).reset_index(drop=True)

In [18]:
# Combine downsampled majority class with upsampled minority class
Balanced_df = pd.concat(
    [
        df_3_upsampled,
        df_4_upsampled,
        df_7_upsampled,
        df_8_upsampled,
        df_5_downsampled,
        df_6_downsampled,
    ]
).reset_index(drop=True)


# Display new class counts
Balanced_df.quality.value_counts()

quality
3    2000
4    2000
7    2000
8    2000
5    2000
6    2000
Name: count, dtype: int64

In [19]:
corr = Balanced_df.corr()["quality"]
predictive_cols = corr[abs(corr) > 0.25].index.tolist()
print(predictive_cols)

['volatile acidity', 'chlorides', 'density', 'alcohol', 'quality']


In [20]:
final_df = Balanced_df[predictive_cols]
final_df = final_df.rename(columns={"volatile acidity": "volatile_acidity"})

In [ ]:
wine_fg = fs.get_or_create_feature_group(
    name="wine",
    version=2,
    primary_key=["volatile_acidity", "chlorides", "density", "alcohol"],
    description="Wine Dataset",
)
wine_fg.insert(final_df)